In [10]:
# Load the filtered clusters CSV

import pandas as pd

CLUSTER_CSV = "../results/filtered_NUMT_candidates.csv"

clusters = pd.read_csv(CLUSTER_CSV)
clusters["num_split_reads"] = 0

print(f"Loaded {len(clusters)} clusters")

Loaded 184 clusters


In [ ]:
# Build a read -> cluster mapping

CLIPPED_READS_CLUSTER_CSV = "../results/clipped_reads_with_cluster.csv"
clipped_df = pd.read_csv(CLIPPED_READS_CLUSTER_CSV)

read_cluster_map = dict(zip(clipped_df["read_name"], clipped_df["cluster_idx"]))

for cluster_idx in clipped_df["cluster_idx"]:
    clusters.at[cluster_idx, "num_split_reads"] += 1

print(clusters.head())

  ref_name  bin_start  is_reverse  num_reads  avg_ref_start  avg_mate_start  \
0        1    1332000        True          2      1332031.0         11714.0   
1        1    6461000        True          2      6461182.0         13807.0   
2        1   16253000       False          2     16253019.0         10256.0   
3        1   18841500       False          2     18841765.0          5177.0   
4        1   51781000       False          2     51781162.0         13806.0   

   num_split_reads  
0               18  
1               11  
2                5  
3                5  
4                1  


In [ ]:
import pysam

MT_BAM = "../results/clipped_vs_mtDNA_sorted.bam"
bam = pysam.AlignmentFile(MT_BAM, "rb")

for read in bam.fetch():
    if read.is_unmapped:
        continue

    cluster_idx = read_cluster_map.get(read.query_name)
    if cluster_idx is not None:
        clusters.at[cluster_idx, "num_split_reads"] += 1

bam.close()

print(clusters[["ref_name", "bin_start", "num_reads", "num_split_reads"]].head())

In [18]:
# Filter clusters again

OUTPUT_CSV = "../results/confirmed_NUMT_clusters.csv"

MIN_SPLIT_READS = 5

confirmed_clusters = clusters[clusters["num_split_reads"] >= MIN_SPLIT_READS]

print(f"Clusters before filtering: {len(clusters)}")
print(f"Clusters after filtering: {len(confirmed_clusters)}")

confirmed_clusters.to_csv(OUTPUT_CSV, index=False)

Clusters before filtering: 184
Clusters after filtering: 88
